
# ЛАБОРАТОРНА РОБОТА  

## "Лінійні та нелінійні регресійні моделі МН"

__Метою__ лабораторної роботи є набуття практичних навичок використання модулів бібліотеки `Scikit-learn` для вирішення наступних задач:

- визначення суттєвих показчиків для регресійної моделі
- пошук та настроювання гіперпараметрів лінійних та нелінійних регресійних моделей

__Результатом__ виконання лабораторної роботи є серія моделей які прогнозують ціну кватрир на вторинному ринку житла

In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('apartment_transformed.csv')
data.shape

(483, 182)

In [2]:
# відокремити ціловий показчик 'price'
price = data.Price

# зилишити в 'data' тільки незалежні показчики
data.drop(columns='Price', inplace=True)

data.shape, price.shape    # ((676, 301), (676,))

((483, 181), (483,))

In [3]:
data

,0,1,2,3,4,5,6,7,8,9,...,171,172,173,174,175,176,177,178,179,180
0,-1.103985,-0.151460,0.357033,-0.511773,0.553360,-0.988970,-1.046168,-1.234396,-0.348154,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.103985,-0.151460,0.357033,-0.511773,0.553360,-0.988970,-1.046168,-1.234396,-0.348154,15.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.151798,-0.126981,1.205132,0.958229,0.395094,-0.287685,-0.132013,-0.173099,-0.266774,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.151798,-0.015636,0.922432,0.618998,0.513794,-0.268205,-0.589091,0.298589,-0.022634,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.151798,-0.128100,-0.491066,0.958229,0.395094,-0.248725,-0.132013,0.298589,-0.321027,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,-0.151798,-0.166567,-1.197816,-1.416390,0.395094,-0.696768,-0.436732,-0.173099,-0.619421,19.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
479,-1.103985,0.059480,-1.056466,-1.190236,0.790761,-0.677288,-1.008079,0.652355,-0.375281,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
480,-1.103985,-0.145725,-0.915116,-1.416390,-1.543675,-0.872089,-0.855719,-0.880630,-0.402407,11.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
481,0.800389,-0.147403,0.498383,0.845152,0.197260,-0.190284,-0.132013,-0.173099,-0.131140,6.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## 1. Пошук значущих ознак

### Теоретичне введення

__Значущі (суттеві) ознаки__ це дані, які мають сильну кореляцію або вплив на результат або прогноз моделі.

Ці позазчики визначаються за допомогою процесу, який називається __відбором ознак__ ([Feature selection](https://en.wikipedia.org/wiki/Feature_selection)), який передбачає _оцінку_ та _ранжування важливості_ різних змінних у наборі даних. Це можна зробити за дпомогою статистичних тестів, кореляційного аналізу або алгоритмів машинного навчання.

Після визначення значущих ознак їх можна використовувати для навчання моделі машинного навчання.

Це може бути особливо важливим у моделях, де дані складні та містять багато змінних. Тому ідентифікація  суттєвих ознак може допомогти зменшити розмірність даних і покращити продуктивність моделі.

Класи в модулі [sklearn.feature_selection](https://scikit-learn.org/stable/modules/feature_selection.html) можна використовувати для вибору функцій/зменшення розмірності на вибіркових наборах або для покращення показників точності оцінювачів, або для підвищення їх продуктивності на масивах даних з дуже великою розмірністю.

### Завдання

Відібрати з вхідного набору `data` 7 найбільш суттєвих показчиків для регрісійної моделі машинного навчання.

In [4]:
# імпортувати з модуля 'feature_selection' селектор ознак 'SelectKBest' 
# та регрісійний тест 'f_regression'
from sklearn.feature_selection import SelectKBest, f_regression

In [5]:
# побудувати селектор 7 ознак на f-регресорі
kbest_selector = SelectKBest(f_regression, k=7)

# застосувати селектор для побудови списку ознак
data_selected = kbest_selector.fit_transform(data, price)

In [6]:
# зберегти імена визначених селектором найбільш суттєвих ознак
best_features = kbest_selector.get_feature_names_out()
best_features

array(['0', '1', '5', '6', '8', '115', '152'], dtype=object)

In [7]:
# побудувати датафрейм на визначених ознаках
data = pd.DataFrame(data=data_selected, columns=best_features)
data.head()

,0,1,5,6,8,115,152
0,-1.103985,-0.151460,-0.988970,-1.046168,-0.348154,0.0,0.0
1,-1.103985,-0.151460,-0.988970,-1.046168,-0.348154,0.0,0.0
2,-0.151798,-0.126981,-0.287685,-0.132013,-0.266774,0.0,0.0
3,-0.151798,-0.015636,-0.268205,-0.589091,-0.022634,0.0,0.0
4,-0.151798,-0.128100,-0.248725,-0.132013,-0.321027,0.0,0.0


### Висновки

_описати загальну статистичну характеристику отриманого датасети та зробити висновки щодо можливості його використання для подальшого аналізу_

...

## 2. Множинна лінійна регресія

### Теоретичне введення

__Множинна лінійна регресія__ — це статистичний метод, який використовується для встановлення зв’язку між _залежною_ (цільвою) змінною $\textbf y$ та _кількома_ незалежними змінними $\textbf [X]$.  

__Метою__ множинної лінійної регресії є знаходження найкращого лінійного зв’язку між залежною змінною та незалежними змінними, який виражається у вигляді рівняння:
$$y = b_0 + b_1 x_1 + b_2 x_2 + ... + b_n x_n$$

Найкращій зв'язок забезпечується знаходженням таких коєфіцієнтів $[B]$, що додають мінімум обраній метриці (MSE, MAE, ...)

### Завдання


Порахувати показчики якості моделі [лінйной множинної регресії](https://uk.mcfairbanks.com/719-multiple-regression-formula) на визначениx п.1 значущих ознаках датасету застосувавши [кросс-валідацію з __10__ сплітами](https://scikit-learn.org/stable/modules/cross_validation.html).

In [8]:
# імпортувати та побудувати лінійний регресор з параметрами за замовчанням
from sklearn.linear_model import LinearRegression
lr = LinearRegression()

In [9]:
# імпортувати крос-валідатор 'cross_validate' з модуля 'model_selection'
from sklearn.model_selection import cross_validate, KFold

In [10]:
# отримати результати крос-валідації по параметрам 'neg_mean_absolute_percentage_error' 
# та 'r2' на 10 сплітах передбачивши розрахунок на навчальному наборі 'return_train_score'
cv_results_mul = cross_validate(lr, data, price, scoring=['neg_mean_absolute_percentage_error', 'r2'],
                               cv=KFold(n_splits=10, shuffle=True), return_train_score=True)

#### занести результати в датафрейм 'cv_results_mul ' наступного вигляду:

------------

|помилка тесту в %   |коєф. R2 тесту  | помилка навчання в %  | коєф. R2 навчання  |
| :------------:|:------------:|:------------:|:------------:|
|  xx.xx | xx.xx  | xx.xx  | xx.xx  |
|  xx.xx | xx.xx  | xx.xx  | xx.xx  |
|  ... | ...  | ...  | ...  |


In [11]:
cv_results_mul = pd.DataFrame({
    'помилка тесту в %': cv_results_mul['test_neg_mean_absolute_percentage_error'],
    'коеф. R2 тесту': cv_results_mul['test_r2'],
    'помилка навчання в %': cv_results_mul['train_neg_mean_absolute_percentage_error'],
    'коеф. R2 навчання': cv_results_mul['train_r2']
})
cv_results_mul.head()

,помилка тесту в %,коеф. R2 тесту,помилка навчання в %,коеф. R2 навчання
0,-0.261286,0.826581,-0.274854,0.995201
1,-0.426788,0.987696,-0.410358,0.986583
2,-0.440339,0.045739,-0.420116,0.988686
3,-0.316431,0.990354,-0.423778,0.986492
4,-0.447410,0.754383,-0.418543,0.987144


In [12]:
# продовжити наступні команди виводу:
print ("середня помилка навчання = ", cv_results_mul['помилка навчання в %'].mean())
print ("середня помилка тесту = ", cv_results_mul['помилка тесту в %'].mean())

середня помилка навчання =  -0.40713964604247943
середня помилка тесту =  -0.4182673107107274


### Висновки

_зпираючись на отримані метрики якості зробити висновок про придатність моделі, недонавчана чи перенавчана вона і т.п._

## 3. Гребнева (Ridge) регресія

### Теоретичне введення

__Гребнева регресія__ — це техніка регулярізації, яка використовується в машинному навчанні для запобігання перенавчанню лінійних регресійних моделей за рахунок додавання штрафу до функції втрат регресійної моделі, яка зменшує величину коефіцієнтів до нуля.

$$\min_w\sum_{i=1}^n(y_i - x_i^w)^2 + \lambda|w|_2^2$$

Розмір штрафу визначається [нормою L2](https://craftappmobile.com/l1-vs-l2-regularization/) вектора коефіцієнтів, помноженою на гіперпараметр $\large \lambda$.




### Завдання

Побудувати модель на основі `ridge-регресії` та за допомогою [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) знайти таке значення _L2-регулярізатора_, яке буде
мінізувати обрані метрики якості моделі.
Для побудови моделі скоистатися датасетом, що отримано в  __завданні 1__ лабораторної роботи


In [13]:
# імпортувати ridge-регресор з модуля `sklearn.linear_model`
from sklearn.linear_model import Ridge

# побудувати регресор
ridge = Ridge(random_state=2, fit_intercept=False)

In [14]:
# імпортувати сітку пошуку `GridSearchCV` з модулю sklearn.model_selection
from sklearn.model_selection import GridSearchCV

# визначити параметр равномірного пошуку 100 значень параметеру `alpha` в диапазоні 0-100000 
grid_params = {
    'alpha': np.arange(0, 10000, 1000)
}

In [15]:
%%time

# побудувати та натренувати гребневу регресійну модель на сітці 'grid_params'
# в якості критерія оцінки якості взяти метрику `neg_mean_absolute_percentage_error`

# створюємо сітку пошуку та тренуємо на ній модель
grid_search_model = GridSearchCV(ridge, 
                                 param_grid=grid_params,
                                 cv=10,
                                 scoring = 'neg_mean_absolute_percentage_error')

grid_search_model.fit(data, price)

CPU times: total: 750 ms
Wall time: 749 ms


GridSearchCV(cv=10, estimator=Ridge(fit_intercept=False, random_state=2),
             param_grid={'alpha': array([   0, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000])},
             scoring='neg_mean_absolute_percentage_error')

In [16]:
# вивести найкращій естіматор (best_estimator_), та найкраще значення обраної метрики (best_score_)
grid_search_model.best_estimator_, grid_search_model.best_score_

(Ridge(alpha=9000, fit_intercept=False, random_state=2), -1.1819236873168844)

### Висновки

_cпираючись на отримані метрики якості зробити висновок про придатність моделі, недонавчана чи перенавчана вона і т.п._

## 3. Поліноміальна регресія

### Теоретичне введення

__Поліноміальна регресія__ — це тип нелінійної регресії, у якому зв’язок між незалежною змінною $\large x$ і залежною змінною $\large y$ моделюється як поліноміальна функція n-го ступеня:

$$ y = \beta_0 + \beta_1 x + \beta_2 x^2 + \beta_3 x^3 + \cdots + \beta_n x^n + \varepsilon $$

__Метою__ поліноміальної регресії є знаходження значень коефіцієнтів $ β_i$, які найкраще відповідають даним.

### Завдання


Порахувати показчики якості моделі на [поліноміальній регресії](https://uk.wikipedia.org/wiki/Поліноміальна_регресія) на визначених в п.1 значущих ознаках датасету, попередньо розширивши датасет за допомогою трансформера [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)

In [17]:
# імпортувати модуль preprocessing.PolynomialFeatures
from sklearn.preprocessing import PolynomialFeatures

# побудувати трансформер ступеня 2 для побудови додаткових ознак в датасеті
poly = PolynomialFeatures(2, include_bias=False)
poly_data = poly.fit_transform(data)

In [18]:
# визначити імена відібраних показчиків
poly_features_names = poly.get_feature_names_out()

In [19]:
# побудувати датасет на визначеному поліномі `poly`
data_poly = pd.DataFrame(poly_data, columns=poly_features_names)
data_poly.shape

(483, 35)

In [20]:
# отримати результати крос-валідації на множинном регресорі `lr` по параметрам 'neg_mean_absolute_percentage_error' 
# та 'r2' на 10 сплітах передбачивши розрахунок на навчальному наборі 'return_train_score'
cv_results_poly = cross_validate(lr,
                                data_poly,
                                price,
                                cv=KFold(n_splits=10, shuffle=True),
                                return_train_score=True,
                                scoring=['neg_mean_absolute_percentage_error', 'r2'])

In [21]:
# занести результати крос-валідації: помилка тесту, помилка навчання та відповідні коефіцієнти 
# детермінаційї в датафрейм `cv_results_poly`. 
cv_results_poly = pd.DataFrame({
    'помилка тесту в %': cv_results_poly['test_neg_mean_absolute_percentage_error'],
    'коеф. R2 тесту': cv_results_poly['test_r2'],
    'помилка навчання в %': cv_results_poly['train_neg_mean_absolute_percentage_error'],
    'коеф. R2 навчання': cv_results_poly['train_r2']
})

cv_results_poly.head()

,помилка тесту в %,коеф. R2 тесту,помилка навчання в %,коеф. R2 навчання
0,-0.175140,0.297607,-0.069299,0.999539
1,-0.112918,-3.964249,-0.084567,0.984416
2,-0.079311,0.996213,-0.081779,0.997574
3,-0.106664,0.947211,-0.073326,0.997766
4,-0.077969,0.980492,-0.080816,0.997618


In [22]:
# за допомогою крос-валідатора 'cross_val_predict' побудувати прогноз 'price_pred' 
# на лінійному регресорі на 10 сплітах
from sklearn.model_selection import cross_val_predict

price_pred = cross_val_predict(lr, data_poly, price, cv=10)

In [23]:
# вивести порівняльну таблицю з двох колонок: ціна реальна, ціна прогнозна
pred = pd.DataFrame({'ціна реальна': price,
                     'ціна прогнозна': price_pred})

pred

,ціна реальна,ціна прогнозна
0,30970.0,26329.061541
1,30970.0,26329.061541
2,82000.0,85229.958525
3,135000.0,145154.248398
4,84000.0,88121.311421
...,...,...
478,42900.0,51619.733995
479,115000.0,117933.571606
480,39000.0,33996.390181
481,115000.0,89250.067268


In [24]:
# натренувати регресор `lr` на поліноміальних ознаках `data_poly`
lr.fit(data_poly, price)

LinearRegression()

In [25]:
# сформувати таблицю коєфіцієнтів поліному
coef = pd.DataFrame({'Ознаки': poly_features_names,
                     'коеф.регресора': lr.coef_.astype('int')})

### Висновки

_Базуючись на значенях метрик абсолютної помилки та r2-оцінки, сформулювати вашу думку чи відповідає поліноміальна модель вимогам якості та дати характеристику декільком коефіцієнтам (3-4) на свій вибір._ 

...

## 5. Зберігання побудованх моделей

In [26]:
# зберегти лінийну, гребневу та поліноміальну моделі у відпрвідних pickle-файлах:
# 'lin_model.pkl', 'ridge_model.pkl', 'poly_model.pkl'

import pickle

names = ['lin_model.pkl', 'ridge_model.pkl', 'poly_model.pkl']
models = [lr, ridge, poly]

for i in range(3):
    with open(names[i], 'wb') as f:
        pickle.dump(models[i], f)